In [22]:
from enum import Enum
from random import shuffle
import numpy as np
from operator import mul
import random
import termcolor
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

class Suit(Enum):
    SPADE = '♠'
    CLUB = '♣'
    HEART = '♡'
    DIAMOND = '♢'
    def __str__(self):
        return self.value
    def __repr__(self):
        return f"Suit.{self.name}"


class Number(Enum):
    ACE = (1, 'A')
    TWO = (2, '2')
    THREE = (3, '3')
    FOUR = (4, '4')
    FIVE = (5, '5')
    SIX = (6, '6')
    SEVEN = (7, '7')
    EIGHT = (8, '8')
    NINE = (9, '9')
    TEN = (10, '10')
    JACK = (11, 'J')
    QUEEN = (12, 'Q')
    KING = (13, 'K')

    def __init__(self, val, string):
        self.val = val
        self.string = string

    def __str__(self):
        return self.string

    def __repr__(self):
        return f"Number.{self.name}"
    
class Card:
    def __init__(self, suit, number):
        if not (isinstance(suit, Suit) and isinstance(number, Number)):
            raise ValueError  # Enum じゃないとエラー
        self.suit = suit
        self.number = number

    def __str__(self):
        return str(self.suit) + str(self.number)

    def __repr__(self):
        return f"Card({self.__str__()})"
    
    def __eq__(self, other):
        return (self.suit, self.number) == (other.suit, other.number)
            
    
class Hand(list):
    def __init__(self,card_list):
        super().__init__(
            i for i in card_list
        )
        
    def check_number(self):
        number_list=[i.number.val for i in self]
        return number_list
    
    def check_suit(self):
        suit_list=[str(i.suit) for i in self]
        return suit_list
    
    def choice(self,card):
        #Card(Suit.SPADE, Number.ACE)
        if card in self:
            self.remove(card)
            return card
        else:
            raise ValueError
            
    def check(self,card):
        return card in self
    

class Deck(list):
    def __init__(self):
        super().__init__(
            Card(suit, number) for suit in Suit for number in Number
        )  # list の初期化を呼び出す
        self.shuffle()  # 最初にシャッフル
    def shuffle(self):
        shuffle(self)
    def draw(self):
        return self.pop()
    def deal(self, players_num):
        cards=[Hand(i) for i in np.array_split(self,players_num)]
        self.clear()
        return cards

In [23]:
# ゲームの状態
class State:
    # 初期化
    def __init__(self, players_num=4,field_cards=None, players_cards=None,turn_player=None,pass_count=None,out_player=None):
        if players_cards==None:
            deck = Deck()
            self.players_cards=deck.deal(players_num)
            self.players_num=players_num
            self.field_cards=np.zeros((4,13), dtype='int64')
            self.start_flags=[0]*self.players_num
            self.pass_count=[0]*self.players_num
            self.out_player=[]
            self.all_cards=[[str(Card(suit, number))  for number in Number] for suit in Suit]
            for players_number in range(players_num):
                self.start_flags[players_number]=self.choice_seven(hand=self.players_cards[players_number])
            self.turn_player=self.start_flags.index(1)
        else:
            self.players_cards=players_cards
            self.field_cards=field_cards
            self.players_num=players_num
            self.turn_player=turn_player
            self.pass_count=pass_count
            self.out_player=out_player
            self.all_cards=[[str(Card(suit, number))  for number in Number] for suit in Suit]
        

    #7のカードを出す
    def choice_seven(self,hand):
        start_flag=0
        for card in [Card(suit,Number.SEVEN) for suit in Suit]:
            if hand.check(Card(Suit.DIAMOND,Number.SEVEN))==True:
                start_flag=1
            if hand.check(card)==True:
                self.put_card(hand.choice(card))
        return start_flag
    
    def choice_card(self,hand,card):
        hand.choice(card)
    
    #場にカードを出す
    def put_card(self,card):
        num=10
        for s,i in zip(Suit,range(4)):
            if card.suit==s:
                num=i
        #state.my_hands().remove(card)
        self.field_cards[num][card.number.val-1]=int(1)
    
    
    # 場で出せる手のリスト取得
    # 3パスの人がいた時、未対応
    def legal_actions(self):
        actions = []
        for suit,n in zip(Suit,range(4)):
            
            if self.field_cards[n][0:6][::-1].tolist().count(1)!=6:
                actions.append(Card(suit,self.num_to_Enum(6-self.field_cards[n][0:6][::-1].tolist().index(0))))
                
            if self.field_cards[n][7:13].tolist().count(1)!=6:
                actions.append(Card(suit,self.num_to_Enum(8+self.field_cards[n][7:13].tolist().index(0))))
        return actions
    
    
    # 自分が出せる手のリスト取得
    def my_actions(self):
        actions = []
        for legal in self.legal_actions():
            if self.players_cards[self.turn_player].check(legal)==True:
                actions.append(legal)
        return actions
    def my_actions_str(self):
        actions = []
        for legal in self.legal_actions():
            if self.players_cards[self.turn_player].check(legal)==True:
                actions.append(legal)
        return [str(i) for i in actions]
    
    # 自分の手札取得
    def my_hands(self):
        return self.players_cards[self.turn_player]
    def my_hands_str(self):
        return [str(i) for i in self.players_cards[self.turn_player]]
            
    
    def num_to_Enum(self,num):
        enum_list=[Number.ACE,Number.TWO,Number.THREE,Number.FOUR, 
                   Number.FIVE,Number.SIX,Number.SEVEN,Number.EIGHT,
                   Number.NINE,Number.TEN,Number.JACK,Number.QUEEN,
                   Number.KING]
        return enum_list[num-1]
    
    
    # 次の状態の取得
    def next(self, action):
        if self.my_actions()==[]:
            self.pass_count[self.turn_player]+=1
            self.pass_check()
        else:
            self.players_cards[self.turn_player].remove(action)
            self.put_card(action)  
        #次のプレイヤーに
        self.next_player() 
        return State(players_num=self.players_num,field_cards=self.field_cards, players_cards=self.players_cards,turn_player=self.turn_player,pass_count=self.pass_count,out_player=self.out_player)
   
    #次のプレイヤーの取得 
    def next_player(self):
        flag=0
        while flag==0:
            if self.turn_player+1>=self.players_num:
                self.turn_player=self.turn_player+1-self.players_num
            else:
                self.turn_player+=1

            if self.turn_player not in self.out_player:
                flag=1
    
    #パスの上限判定
    def pass_check(self):
        out_list=self.out_player
        if self.pass_count[self.turn_player]>3:
            for card in self.my_hands():
                self.put_card(card)
            out_list.append(self.turn_player)
            
            self.out_player=out_list
            
    def to_str(self,num):
        return str(num)
    
    #勝ち負け判定
    def is_done(self):
        return len(self.my_hands())==0
        
            
    # 状態表示
    def __str__(self):
        str = ''
        field_cards=self.field_cards.tolist()
        out_list=[list(map(mul,self.all_cards[i],field_cards[i])) for i in range(4)]
        str += "場のカード\n\n"
        for i in range(len(out_list)):
            minilist=out_list[i]
            for j in range(len(minilist)):
                if minilist[j] == "":
                    str += " -- "
                else:
                    str +=" "+minilist[j]+" "
            str += '\n'
        num=self.to_str(self.turn_player)
        pass_cnt=self.to_str(self.pass_count[self.turn_player])
        str+="\nプレイヤー"+num+"番　　パス回数"+pass_cnt+"\n"
        str += "\nあなたの手札\n"
        
        out_list=self.my_hands_str()
        for i in range(len(out_list)):
            str+=out_list[i]
            str+=" "
            
        str += "\n\n出せるカード\n"
        
        out_list=self.my_actions_str()
        for i in range(len(out_list)):
            str+=out_list[i]
            str+=" "
        
        str += "\n"
        
        return str    

In [24]:
def num_to_Card(number,suit):
    number_list=[Number.ACE,Number.TWO,Number.THREE,Number.FOUR, 
                Number.FIVE,Number.SIX,Number.SEVEN,Number.EIGHT,
                Number.NINE,Number.TEN,Number.JACK,Number.QUEEN,
                Number.KING]
    suit_list=[Suit.SPADE,Suit.CLUB,Suit.HEART,Suit.DIAMOND]
    return Card(suit_list[suit],number_list[number-1])

In [25]:
# ランダム行動 AI
def random_action(state):
    my_actions = state.my_actions()
    if my_actions != []:
        return my_actions[random.randint(0, len(my_actions)-1)]
    else:
        my_actions=[]
        return my_actions

In [26]:
import random

def best_action(state):
    my_actions = state.my_actions()
    if not my_actions:
        # No available actions
        return []
    else:
        suit_to_index = {'♠': 0, '♣': 1, '♡': 2, '♢': 3}
        best_score = float('-inf')
        best_cards = []

        # Track the number of cards in each suit
        suit_counts = {suit: 0 for suit in suit_to_index.keys()}
        for card in state.my_hands():
            suit_counts[card.suit.value] += 1

        for card in my_actions:
            suit = card.suit.value
            suit_index = suit_to_index[suit]
            number_index = card.number.val - 1  # 0-based index
            score = 0

            # Prioritize Aces and Kings
            if number_index == 0 or number_index == 12:
                score += 5  # Higher weight to avoid opening new cards

            # Check adjacent cards
            next_indices = []
            if number_index < 6:
                next_indices.append(number_index - 1)
            elif number_index > 6:
                next_indices.append(number_index + 1)

            for next_number_index in next_indices:
                if 0 <= next_number_index <= 12:
                    if state.field_cards[suit_index][next_number_index] == 1:
                        score += 2  # Next card is already on the table
                    else:
                        # Penalty for potentially opening cards for opponents
                        score -= 3
                        # Reduce penalty if you have the next card
                        have_next_card = any(
                            c for c in state.my_hands()
                            if c.suit.value == suit and c.number.val - 1 == next_number_index
                        )
                        if have_next_card:
                            score += 2  # You can control the next move

            # Consider the number of cards you have in this suit
            score += suit_counts[suit] * 0.5  # Encourage playing suits where you have more cards

            # Favor reducing hand size
            score += (len(state.my_hands()) - 1) * 0.1

            # Determine if this move opens up new moves for you
            potential_new_moves = 0
            for c in state.my_hands():
                if c.suit.value == suit:
                    c_index = c.number.val - 1
                    if (number_index < 6 and c_index == number_index - 1) or \
                       (number_index > 6 and c_index == number_index + 1):
                        potential_new_moves += 1
            score += potential_new_moves * 1.5  # High weight for opening your own cards

            # Decide on the best card
            if score > best_score:
                best_score = score
                best_cards = [card]
            elif score == best_score:
                best_cards.append(card)

        # Randomly choose among the best cards
        return random.choice(best_cards)


In [27]:
#課題
#ここに自分のAIを作成して下さい
import random

def my_AI(state):
    my_actions = state.my_actions()
    if not my_actions:
        # No available actions
        return []
    else:
        suit_to_index = {'♠': 0, '♣': 1, '♡': 2, '♢': 3}
        best_score = float('-inf')
        
        best_cards = []

        # Track the number of cards in each suit
        suit_counts = {suit: 0 for suit in suit_to_index.keys()}
        for card in state.my_hands():
            suit_counts[card.suit.value] += 1

        for card in my_actions:
            suit = card.suit.value
            suit_index = suit_to_index[suit]
            number_index = card.number.val - 1  # 0-based index
            score = 0

            # Prioritize Aces and Kings
            if number_index == 0 or number_index == 12:
                score += 5  # Higher weight to avoid opening new cards

            # Check adjacent cards
            next_indices = []
            if number_index < 6:
                next_indices.append(number_index - 1)
            elif number_index > 6:
                next_indices.append(number_index + 1)

            for next_number_index in next_indices:
                if 0 <= next_number_index <= 12:
                    if state.field_cards[suit_index][next_number_index] == 1:
                        score += 2  # Next card is already on the table
                    else:
                        # Penalty for potentially opening cards for opponents
                        score -= 3
                        # Reduce penalty if you have the next card
                        have_next_card = any(
                            c for c in state.my_hands()
                            if c.suit.value == suit and c.number.val - 1 == next_number_index
                        )
                        if have_next_card:
                            score += 2  # You can control the next move

            # Consider the number of cards you have in this suit
            score += suit_counts[suit] * 0.5  # Encourage playing suits where you have more cards

            # Favor reducing hand size
            score += (len(state.my_hands()) - 1) * 0.1

            # Determine if this move opens up new moves for you
            potential_new_moves = 0
            for c in state.my_hands():
                if c.suit.value == suit:
                    c_index = c.number.val - 1
                    if (number_index < 6 and c_index == number_index - 1) or \
                       (number_index > 6 and c_index == number_index + 1):
                        potential_new_moves += 1
            score += potential_new_moves * 1.5  # High weight for opening your own cards

            # Decide on the best card
            if score > best_score:
                best_score = score
                best_cards = [card]
            elif score == best_score:
                best_cards.append(card)

        # Randomly choose among the best cards
        return random.choice(best_cards)


In [28]:
# ランダムAIと対戦
state = State()
# ゲーム終了までのループ
while True:
    # ゲーム終了時
    if state.is_done():
        print("勝者 プレイヤー"+str(state.turn_player)+"番")
        break;
        
    # 行動の取得
    if state.turn_player==0:
        action = my_AI(state)
        print(termcolor.colored(state, 'red'))
    else:
        action = best_action(state)
        print(state)

    # 次の状態の取得
    state = state.next(action)

AttributeError: 'State' object has no attribute 'all_actions'

In [8]:
# ランダムAIとの勝率チェック
# パラメータ
EP_GAME_COUNT = 2000  # 1評価あたりのゲーム数

def player_point(ended_state):
    #print(termcolor.colored(ended_state.turn_player, 'red'))
    if ended_state.turn_player==0:
        return 1
    return 0

def play(next_actions):
    state = State()
    while True:
        if state.is_done():
            break
        if state.turn_player==0:
            action = my_AI(state)
        else:
            action = random_action(state)
        state = state.next(action)      
    return player_point(state)

# 任意のアルゴリズムの評価
def evaluate_algorithm_of(label, next_actions):
    # 複数回の対戦を繰り返す
    total_point = 0
    for i in range(EP_GAME_COUNT):
        total_point += play(next_actions)
        print('\rEvaluate {}/{}'.format(i + 1, EP_GAME_COUNT), end='')
    print('')

    # 平均ポイントの計算
    average_point = total_point / EP_GAME_COUNT
    print(label.format(average_point))

# VSランダム
next_actions = (random_action, random_action)
evaluate_algorithm_of('VS_Random {:.3f}', next_actions)

Evaluate 2000/2000
VS_Random 0.384


In [9]:
state=State()

In [10]:
print(state)

場のカード

 --  --  --  --  --  --  ♠7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♣7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♡7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♢7  --  --  --  --  --  -- 

プレイヤー0番　　パス回数0

あなたの手札
♡3 ♢3 ♡Q ♡2 ♠2 ♢2 ♢10 ♡K ♠K ♠5 ♣A ♠4 

出せるカード




### 1.手札に関する関数

In [11]:
 #リストで手札を表示する　
state.my_hands()

[Card(♡3),
 Card(♢3),
 Card(♡Q),
 Card(♡2),
 Card(♠2),
 Card(♢2),
 Card(♢10),
 Card(♡K),
 Card(♠K),
 Card(♠5),
 Card(♣A),
 Card(♠4)]

In [12]:
#リストで手札の数字を表示する
state.my_hands().check_number()

[3, 3, 12, 2, 2, 2, 10, 13, 13, 5, 1, 4]

In [13]:
#リストで手札のマークを表示する
state.my_hands().check_suit()

['♡', '♢', '♡', '♡', '♠', '♢', '♢', '♡', '♠', '♠', '♣', '♠']

In [14]:
#リストで自分が出せるカードを表示する
state.my_actions()

[]

In [15]:
#リストで自分が出せるカードの数字を表示する
Hand(state.my_actions()).check_number()

[]

In [16]:
#リストで自分が出せるカードの記号を表示する
Hand(state.my_actions()).check_suit()

[]

### 2.場の札に関する関数

In [17]:
#場のカードを表示する
state.field_cards

array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [18]:
#場で出せるカードをリストで取得する
state.legal_actions()

[Card(♠6),
 Card(♠8),
 Card(♣6),
 Card(♣8),
 Card(♡6),
 Card(♡8),
 Card(♢6),
 Card(♢8)]

In [19]:
#場で出せるカードの数字をリストで取得する
Hand(state.legal_actions()).check_number()

[6, 8, 6, 8, 6, 8, 6, 8]

In [20]:
#場で出せるカードの記号をリストで取得する
Hand(state.legal_actions()).check_suit()

['♠', '♠', '♣', '♣', '♡', '♡', '♢', '♢']

 ### 3.状態に関する関数

In [21]:
#今のプレイヤーの番号を表示する
state.turn_player

0

In [22]:
#3回パスをしてしまったプレイヤーを表示する
state.out_player

[]

### 4.pythonプログラミングの基礎

#### print

何かを表示するときはprintというものを使います。

In [23]:
print("Hello World")
print(5)

Hello World
5


#### 計算式

四則演算ができます。

In [24]:
print(5+4)  #足し算
print(5-4)  #引き算
print(5*4)  #掛け算
print(5/4)  #割り算
print(5%4)  #割った余りを求める

9
1
20
1.25
1


#### 変数

数学の文字と同じで数字を代入することができます。文章も代入できます。

In [25]:
a = 3
b = 1+3  #計算式の形で代入ができます
c = "こんにちは"  #文字列も代入できます
aisatsu = "こんばんは"  #変数名は何文字でもいいです
print(a)
print(b)
print(c)
print(aisatsu)

3
4
こんにちは
こんばんは


#### リスト

リストを使うとたくさんの数字や文字をまとめることができます。数学の添字と同じです。

In [26]:
l = [1,2,3,4,5]
print(l[3])  #0番目から数えて3番目の要素を返します

l.append(100)  #末尾に100を追加します
print(l)

4
[1, 2, 3, 4, 5, 100]


#### if文

if文を使うことで条件分岐をすることができます。

#### for文

for文を使うことで繰り返し処理をすることができます。

In [27]:
#:とインデントを忘れないようにしてください。
for i in range(5):
    print("Hello World.")

Hello World.
Hello World.
Hello World.
Hello World.
Hello World.


こんな使い方もあります。

In [28]:
for item in ["Apple", "Orange", "Banana", "Melon"]:
    print(item)

Apple
Orange
Banana
Melon


#### 関数

関数というものを使うと何度も使う機能を少ないコーディングで書くことができます。数学の関数と同じで値を入力すると値を計算して返します。

In [29]:
#:とインデントを忘れないようにしてください。
#引数を3乗する関数
def testfunc(hikisu):
    cube = hikisu*hikisu*hikisu
    return cube

In [30]:
print(testfunc(3))

27


### 4.便利な関数

In [31]:
#リストを定義する
l = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [32]:
#最大値を求める関数
max(l)

13

In [33]:
#最小値を求める関数
min(l)

1

In [34]:
#昇順に並び替える関数
sorted(l)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [35]:
#降順に並び替えるときはreverse = Trueをつける
sorted(l, reverse = True)

[13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

In [36]:
#7からの距離を求める関数
def DistFrom7(num):
    return abs(num-7)

In [37]:
#keyを使うと7からの距離で最大最小並び替えができる
print(max(l, key = DistFrom7))
print(min(l, key = DistFrom7))
print(sorted(l, key = DistFrom7))

1
7
[7, 6, 8, 5, 9, 4, 10, 3, 11, 2, 12, 1, 13]


In [38]:
#使用例
#手札を7から近い順に並び替える
def DistFrom7(hand):
    return abs(hand.number.val-7)


hands_sorted = sorted(state.my_hands(), key = DistFrom7, reverse = False)
print(state.my_hands())
print(hands_sorted)

[Card(♡3), Card(♢3), Card(♡Q), Card(♡2), Card(♠2), Card(♢2), Card(♢10), Card(♡K), Card(♠K), Card(♠5), Card(♣A), Card(♠4)]
[Card(♠5), Card(♢10), Card(♠4), Card(♡3), Card(♢3), Card(♡Q), Card(♡2), Card(♠2), Card(♢2), Card(♡K), Card(♠K), Card(♣A)]


### 5.Classの説明
この7ならべプログラムではclassと呼ばれる概念が使われています。classを使うと「もの」をわかりやすく記述することができます。

たとえば、身長と好きな色がある「人」というクラスを作ってみます。

In [39]:
class person:
    height = 0
    favcolor = "hoge"

これで「人」が定義できました。それでは田中さんを作ってみます。

In [40]:
tanaka = person()

tanaka.heightとすると身長が、tanaka.colorとすると好きな色が表示できます。しかし田中さんの身長と好きな色はまだ初期のままです。

In [41]:
print(tanaka.height)
print(tanaka.favcolor)

0
hoge


田中さんの身長と好きな色を代入してみましょう

In [42]:
tanaka.height=150
tanaka.favcolor="blue"

In [43]:
print(tanaka.height)
print(tanaka.favcolor)

150
blue


classの中には関数を入れることもできます。ためしに身長と好きな色を表示する関数を作ってみます。<br>関数内で変数を扱うときは「そのクラス自身の変数」であることをいうためにself.heightのようにします。

In [44]:
class person:
    height = 0
    favcolor = "hoge"
    
    def explain(self):
        print("身長は"+str(self.height)+"、好きな色は"+self.favcolor)

In [45]:
okada = person()
okada.height = 160
okada.favcolor = "pink"

In [46]:
okada.explain()

身長は160、好きな色はpink


\_\_init\_\_という関数はclassを代入したときに自動的に動く関数です。これを使うと最初に変数を代入するときなどに便利です。

In [47]:
class person:
    def __init__(self, height, favcolor):
        self.height = height
        self.favcolor = favcolor
    
    def explain(self):
        print("身長は"+str(self.height)+"、色は"+self.favcolor)

In [48]:
suzuki = person(170, "yellow")

In [49]:
suzuki.explain()

身長は170、色はyellow
